In [1]:
import cv2
import itertools
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def detectFacialLandmarks(image, face_mesh):
    '''
    This function performs facial landmarks detection on an image.
    Args:
        image:     The input image of person(s) whose facial landmarks needs to be detected.
        face_mesh: The face landmarks detection function required to perform the landmarks detection.
    Returns:
        output_image: A copy of input image with face landmarks drawn.
        results:      The output of the facial landmarks detection on the input image.
    '''
    
    # Perform the facial landmarks detection on the image, after converting it into RGB format.
    # TODO: check why we have to put ::-1
    results = face_mesh.process(image[:,:,::-1])
    
    #---------- PRINT THE LANDMARKS ON THE IMAGE ----------
    
    # Create a copy of the input image to draw facial landmarks.
    output_image = image[:,:,::-1].copy()
    
    # Check if facial landmarks in the image are found.
    if results.multi_face_landmarks:

        # Iterate over the found faces.
        for face_landmarks in results.multi_face_landmarks:

            # Draw the facial landmarks on the output image with the face mesh tesselation
            # connections using default face mesh tesselation style.
            mp_drawing.draw_landmarks(image=output_image, landmark_list=face_landmarks,
                                      connections=mp_face_mesh.FACEMESH_TESSELATION,
                                      landmark_drawing_spec=None, 
                                      connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

            # Draw the facial landmarks on the output image with the face mesh contours
            # connections using default face mesh contours style.
            mp_drawing.draw_landmarks(image=output_image, landmark_list=face_landmarks,
                                      connections=mp_face_mesh.FACEMESH_CONTOURS,
                                      landmark_drawing_spec=None, 
                                      connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
            
    #------------------------------------------------------
    
        
    # Return the output image in BGR format and results of facial landmarks detection.
    # TODO: check why we have to put ::-1
    return np.ascontiguousarray(output_image[:,:,::-1], dtype=np.uint8), results

In [3]:
def getSize(image, face_landmarks, INDEXES):
    '''
    This function calculates the height and width of a face part utilizing its landmarks.
    Args:
        image:          The image of person(s) whose face part size is to be calculated.
        face_landmarks: The detected face landmarks of the person whose face part size is to 
                        be calculated.
        INDEXES:        The indexes of the face part landmarks, whose size is to be calculated.
    Returns:
        width:     The calculated width of the face part of the face whose landmarks were passed.
        height:    The calculated height of the face part of the face whose landmarks were passed.
        landmarks: An array of landmarks of the face part whose size is calculated.
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape
    
    # Initialize a list to store the landmarks of the face part.
    landmarks = []
    
    # Iterate over the indexes of the landmarks of the face part. 
    for INDEX in INDEXES:
        
        # Append the landmark into the list.
        landmarks.append([int(face_landmarks.landmark[INDEX].x * image_width),
                               int(face_landmarks.landmark[INDEX].y * image_height)])
    
    # Calculate the width and height of the face part.
    # TODO: Transform it to take into account the shearing --> find the 3D boundinx box and project 
    # the closest face to the screen
    _, _, width, height = cv2.boundingRect(np.array(landmarks))
    

    # Convert the list of landmarks of the face part into a numpy array.
    landmarks = np.array(landmarks)
    
    # Return the calculated width height and the landmarks of the face part.
    return width, height, landmarks

In [4]:
def isOpen(image, face_mesh_results, face_part, threshold=5):
    '''
    This function checks whether the eye or mouth of the person(s) is open, 
    utilizing its facial landmarks.
    Args:
        image:             The image of person(s) whose an eye or mouth is to be checked.
        face_mesh_results: The output of the facial landmarks detection on the image.
        face_part:         The name of the face part that is required to check: MOUTH, RIGHT EYE, LEFT EYE
        threshold:         The threshold value used to check the isOpen condition.
    Returns:
        status:       A dictionary containing isOpen statuses of the face part of all the 
                      detected faces.  
    '''
    
    # Retrieve the height and width of the image.
    image_height, image_width, _ = image.shape

    
    # Create a dictionary to store the isOpen status of the face part of all the detected faces.
    status={}
    
    # Check if the face part is mouth.
    if face_part == 'MOUTH':
        
        # Get the indexes of the mouth.
        INDEXES = mp_face_mesh.FACEMESH_LIPS
        
    # Check if the face part is left eye.    
    elif face_part == 'LEFT EYE':
        
        # Get the indexes of the left eye.
        INDEXES = mp_face_mesh.FACEMESH_LEFT_EYE
    
    # Check if the face part is right eye.    
    elif face_part == 'RIGHT EYE':
        
        # Get the indexes of the right eye.
        INDEXES = mp_face_mesh.FACEMESH_RIGHT_EYE 
           
    # Otherwise return nothing.
    else:
        return
    
    # Convert the indexes of the landmarks of the face part into a list.
    # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
    INDEXES_LIST = set(list(itertools.chain(*INDEXES)))
    
    # Iterate over the found faces.
    for face_no, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
        
        # Get the height of the face part.
        _, height, _ = getSize(image, face_landmarks, INDEXES_LIST)
        
        # Get the height of the whole face.
        face_oval = set(list(itertools.chain(*mp_face_mesh.FACEMESH_FACE_OVAL)))
        _, face_height, _ = getSize(image, face_landmarks, face_oval)
        
        # Check if the face part is open.
        if (height/face_height)*100 > threshold:
            
            # Set status of the face part to open.
            status[face_no] = 'OPEN'
        
        # Otherwise.
        else:
            # Set status of the face part to close.
            status[face_no] = 'CLOSE'
    
    # Otherwise
    else:
        
        # Return the output image and the isOpen statuses of the face part of each detected face.
        return status

In [5]:
def checkRotation(face_landmarks):
    face_landmarks_l = []
    for index in range(len(face_landmarks.landmark)):
        face_landmarks_l.append([face_landmarks.landmark[index].x, 
                          face_landmarks.landmark[index].y, 
                          face_landmarks.landmark[index].z])
    
    o3d_landmarks_f = o3d.utility.Vector3dVector(np.array(face_landmarks_l))
    o3d_bbox_f = o3d.geometry.OrientedBoundingBox.create_from_points(o3d_landmarks_f)
    
    R = o3d_bbox_f.R
    points = np.asarray(o3d_bbox_f.get_box_points())
    
    # After projection the points stored with this indexes
    #  2*****7
    #  *     *
    #  *     *
    #  0*****1
    # So we check how they turned in local space to get the rotation/inversion
    if points[2, 0] > points[1, 0] and points[2, 1] > points[1, 1]:
        return 0, R, points
    elif points[2, 0] < points[1, 0] and points[2, 1] < points[1, 1]:
        return 1, R, points
    elif points[2, 0] < points[1, 0] and points[2, 1] > points[1, 1]:
        return 2, R, points
    elif points[2, 0] > points[1, 0] and points[2, 1] < points[1, 1]:
        return 3, R, points
    else:
        return 4, R, points

In [6]:
def boundingBox(face_landmarks, face_part, INDEXES, angle, R, required_size):
    
    landmarks = []
    for index in INDEXES:
        landmarks.append([face_landmarks.landmark[index].x, 
                          face_landmarks.landmark[index].y, 
                          face_landmarks.landmark[index].z])
    
    # Create the oriented bounding box
    o3d_landmarks = o3d.utility.Vector3dVector(np.array(landmarks))
    o3d_bbox = o3d.geometry.OrientedBoundingBox.create_from_points(o3d_landmarks)
    
    # Get box points and center point
    box_points = np.asarray(o3d_bbox.get_box_points())
    box_center = np.asarray(o3d_bbox.get_center())

    
    # Project in global space and center 
    R_inv = np.linalg.inv(R)
    
    
    # One of the forward point
    # We only need to recover the global depth to project the filter image on the forward rectangle
    print(box_points[:,2])
    index = np.argmin(box_points[:,2])
    forward_point = box_points[index, :] 
    forward_point_projected = np.matmul(R_inv, forward_point - box_center)

    
    # Apply transformations (depends on the filter type)
    required_width, required_height = required_size
    forward_points_transformed = []

    # Deal with the position depending on the filter type AND the orientation of the face bounding box
    if face_part == 'FOREHEAD':        
        if angle == 0:            
            forward_points_transformed.append([required_height, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, required_width / 2, forward_point_projected[2]])
        elif angle == 1:
            forward_points_transformed.append([-required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, - required_width / 2, forward_point_projected[2]])
        elif angle == 2:
            forward_points_transformed.append([required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, - required_width / 2, forward_point_projected[2]])
        elif angle == 3:
            forward_points_transformed.append([-required_height, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, required_width / 2, forward_point_projected[2]])
        else:
            return

            
    elif face_part == 'RIGHT EYE' or face_part == 'LEFT EYE':  
        if angle == 0:            
            forward_points_transformed.append([required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
        elif angle == 1:
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, - required_width / 2, forward_point_projected[2]])
        elif angle == 2:
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, - required_width / 2, forward_point_projected[2]])
        elif angle == 3:
            forward_points_transformed.append([-required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height/2, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height/2, required_width / 2, forward_point_projected[2]])
        else:
            return
        
    elif face_part == 'MOUTH':
        if angle == 0:            
            forward_points_transformed.append([0, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
        elif angle == 1:
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, - required_width / 2, forward_point_projected[2]])
        elif angle == 2:
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([-required_height, - required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, - required_width / 2, forward_point_projected[2]])
        elif angle == 3:
            forward_points_transformed.append([0, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, -required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([required_height, required_width / 2, forward_point_projected[2]])
            forward_points_transformed.append([0, required_width / 2, forward_point_projected[2]])
        else:
            return     
    else:
        return
       
    # Project the filter image in the local space
    forward_points = []
    for p in forward_points_transformed:
        p_rotated = np.matmul(R, p) # Rotate to local space
        p_translated = p_rotated + box_center # Translate to box position
        forward_points.append(p_translated)
            
    return o3d_bbox, box_center, forward_points


In [7]:
def drawFilter(image, filter_img, face_landmarks, face_part, INDEXES, angle, R, required_size):
    
    filter_img_height, filter_img_width, _  = filter_img.shape
    image_height, image_width, _ = image.shape
    
    required_width = required_size[0]
    required_height = required_size[1]
    
    # Get 3d bounding box and fit the filter image in it
    required_height_norm = required_height / image_height
    required_width_norm = required_width / image_width
    
    # Get the bounding box of the right size
    o3d_bbox, box_center_array, forward_points = boundingBox(face_landmarks, face_part, INDEXES, angle, R,
                                                             [required_width_norm, required_height_norm])
    #print(forward_points)
          
    # Get the initial corner points of the filter image (counterclockwise)
    pts1 = np.array([[0, 0], [0, filter_img_height], 
                     [filter_img_width, filter_img_height], [filter_img_width, 0]])
    
    # Get the final corner points of the filter image in the overlayered image (counterclockwise)
    # TODO: mettre en perspective au lieu de orthographic
    pts2 = []
    for point in forward_points:
        resolution_x = 1
        resolution_y = 1
        fov = 45
        z0 = (resolution_x / 2) / np.tan((fov/2) * np.pi / 180)
        
        #print(point[0])
        #print(point[0] * z0 / (z0 + point[2]))
        
        
        relative_x = int((point[0] * z0 / (z0 + point[2])) * image.shape[1])
        relative_y = int((point[1] * z0 / (z0 + point[2])) * image.shape[0])

        pts2.append([relative_x, relative_y])
    pts2 = np.array(pts2)

    # Find homography from the filter image to the overlayed image
    h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)

    # Put the filter image in perspective
    im1Reg = cv2.warpPerspective(filter_img, h, (image_width, image_height))

    # Inverse filter image mask in perspective
    _, filter_img_mask = cv2.threshold(cv2.cvtColor(im1Reg, cv2.COLOR_BGR2GRAY),
                                   25, 255, cv2.THRESH_BINARY_INV)

    # Spread in 3 dimensions (rgb)
    filter_img_mask = np.expand_dims(filter_img_mask, axis=2)
    filter_img_mask = np.repeat(filter_img_mask, 3, axis=2)

    # Final image with black pixels at filter image position
    resultant_image = cv2.bitwise_and(image, filter_img_mask)

    # Use Bitwise or to merge the two images
    annotated_image = cv2.bitwise_or(im1Reg, resultant_image)        

    # DEBUG
    box_points = np.asarray(o3d_bbox.get_box_points())

    R = o3d_bbox.R
    R_inv = np.linalg.inv(R)
    
    points_projected = []
    for p in box_points:
        points_projected.append(np.matmul(R_inv, p - box_center_array))
    points_projected = np.array(points_projected)
    #print(points_projected)
    
    relative_x = int((box_center_array[0] * z0 / (z0 + box_center_array[2])) * frame.shape[1])
    relative_y = int((box_center_array[1] * z0 / (z0 + box_center_array[2])) * frame.shape[0])

    cv2.circle(annotated_image, (relative_x, relative_y), radius=1, color=(0, 0, 255), thickness=7)
    
    i = 0
    #for point in box_points[[3, 4, 5, 6], :]:
    for point in box_points[[0, 1, 2, 7], :]:
        relative_x = int((point[0] * z0 / (z0 + point[2])) * frame.shape[1])
        relative_y = int((point[1] * z0 / (z0 + point[2])) * frame.shape[0])

        #if point[2] < 0:
        #    color = (255, 0, 0) # négatif c'est devant !
        #else:
        #    color = (0, 255, 0)
        cv2.circle(annotated_image, (relative_x, relative_y), radius=1, color=(0, 0+i, 0), thickness=7)
        i+= 70

    return annotated_image

In [8]:
# Initialize the mediapipe face mesh class.
mp_face_mesh = mp.solutions.face_mesh

# Setup the face landmarks function for videos.
# TODO: Here works for only one face --> Make it work for multiple faces (see overlay call in main function)
face_mesh_videos = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, 
                                         min_detection_confidence=0.5,min_tracking_confidence=0.3)

In [9]:
# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

# Initialize the mediapipe drawing styles class.
mp_drawing_styles = mp.solutions.drawing_styles

In [10]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)

# Set camera resolution
camera_video.set(3,1280)
camera_video.set(4,960)

# Create named window for resizing purposes.
cv2.namedWindow('Face Filter', cv2.WINDOW_NORMAL)

# Read the left and right eyes images.
# TODO: Create an interactive window to let the user select the wanted filters
left_eye = cv2.imread('data/left_eye_cupcake.png')
right_eye = cv2.imread('data/right_eye_cupcake.png')

crown = cv2.imread('data/crown.png')

# Initialize the VideoCapture object to read from the smoke animation video stored in the disk.
# TODO: Same as for images --> Create a dictionary to store all images and videos?
animation = cv2.VideoCapture('data/rainbow_animation1.mp4')

# Set the smoke animation video frame counter to zero.
animation_frame_counter = 0

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then continue to the next iteration to read
    # the next frame.
    if not ok:
        continue
        
    # Get the width and height of the image
    image_height, image_width, _ = frame.shape
        
    # Read a frame from smoke animation video
    _, animation_frame = animation.read()
    
    # Increment the smoke animation video frame counter.
    animation_frame_counter += 1
    
    # Check if the current frame is the last frame of the animation video.
    if animation_frame_counter == animation.get(cv2.CAP_PROP_FRAME_COUNT):     
        
        # Set the current frame position to first frame to restart the video.
        animation.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
        # Set the animation video frame counter to zero.
        animation_frame_counter = 0
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Perform Face landmarks detection.
    _, face_mesh_results = detectFacialLandmarks(frame, face_mesh_videos)
    
    
    # Check if facial landmarks are found.
    if face_mesh_results.multi_face_landmarks:        
        # Get the mouth isOpen status of the person in the frame.
        mouth_status = isOpen(frame, face_mesh_results, 'MOUTH', 
                                     threshold=15)
        
        # Get the left eye isOpen status of the person in the frame.
        left_eye_status = isOpen(frame, face_mesh_results, 'LEFT EYE', 
                                        threshold=4.5)
        
        # Get the right eye isOpen status of the person in the frame.
        right_eye_status = isOpen(frame, face_mesh_results, 'RIGHT EYE', 
                                         threshold=4.5)
        
        # Iterate over the found faces.
        for face_num, face_landmarks in enumerate(face_mesh_results.multi_face_landmarks):
            
            angle, R, points = checkRotation(face_landmarks)
            
            # Check if the left eye of the face is open.
            if left_eye_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = left_eye.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                left_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
                
                            # Get the height of the face part on which we will overlay the filter image.
                _, face_part_height, landmarks = getSize(frame, face_landmarks, left_eye_landmarks)

                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
                # of the eye
                required_height = int(face_part_height*2.5)
                required_width = int(filter_img_width*(required_height/filter_img_height))
                
                frame = drawFilter(frame, left_eye, face_landmarks, 'LEFT EYE',
                                   left_eye_landmarks, angle, R, [required_width, required_height])
                
            
            # Check if the right eye of the face is open.
            if right_eye_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = right_eye.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                right_eye_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))
                
                # Get the height of the face part on which we will overlay the filter image.
                _, face_part_height, landmarks = getSize(frame, face_landmarks, right_eye_landmarks)

                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
                # of the eye
                required_height = int(face_part_height*2.5)
                required_width = int(filter_img_width*(required_height/filter_img_height))
                
                frame = drawFilter(frame, right_eye, face_landmarks, 'RIGHT EYE',
                                   right_eye_landmarks, angle, R, [required_width, required_height])
                            
            # Check if the mouth of the face is open.
            if mouth_status[face_num] == 'OPEN':
                
                # Get the width and height of filter image.
                filter_img_height, filter_img_width, _  = animation_frame.shape
                
                # Convert the indexes of the landmarks of the face part into a list.
                # TODO: Rewrite it in a more natural way OR undestand it + Is it necessaray to create it as a list?
                lips_landmarks = set(list(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))
                
                # Get the height of the face part on which we will overlay the filter image.
                face_part_width, face_part_height, landmarks = getSize(frame, face_landmarks, lips_landmarks)
                
                # Specify the height to which the filter image is required to be resized.
                # 2.5 can be changed depending on the size of the filter we want
                # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
                # of the eye
                #required_height = int(face_part_height*1.5)
                #required_width = int(filter_img_width*(required_height/filter_img_height))
                required_width = int(face_part_width*0.7)
                required_height = int(filter_img_height*(required_width/filter_img_width))
                
                frame = drawFilter(frame, animation_frame, face_landmarks, 'MOUTH', 
                                   lips_landmarks, angle, R, [required_width, required_height])
            
            # FOREHEAD
            # Get the width and height of filter image.
            filter_img_height, filter_img_width, _  = crown.shape
            
            forehead_landmarks = [103, 67, 109, 10, 338, 297, 332]
            
            # Get the height of the face part on which we will overlay the filter image.
            face_part_width,_, landmarks = getSize(frame, face_landmarks, forehead_landmarks)
                
            # Specify the height to which the filter image is required to be resized.
            # 2.5 can be changed depending on the size of the filter we want
            # This allows the filter to be bigger/smaller depending on both the size of the eye and the size of the aperture
            # of the eye
            required_width = int(face_part_width*2.5)
            required_height = int(filter_img_height*(required_width/filter_img_width))
            
            frame = drawFilter(frame, crown, face_landmarks, 'FOREHEAD',
                                   forehead_landmarks, angle, R, [required_width, required_height])
            
            i = 0
            #for point in box_points[[3, 4, 5, 6], :]:
            for point in points[[0, 1, 2, 7], :]:
                resolution_x = 1
                resolution_y = 1
                fov = 45
                z0 = (resolution_x / 2) / np.tan((fov/2) * np.pi / 180)
        
                relative_x = int((point[0] * z0 / (z0 + point[2]))* frame.shape[1])
                relative_y = int((point[1] * z0 / (z0 + point[2])) * frame.shape[0])

                #if point[2] < 0:
                #    color = (255, 0, 0) # négatif c'est devant !
                #else:
                #    color = (0, 255, 0)
                cv2.circle(frame, (relative_x, relative_y), radius=1, color=(0, 0+i, 0), thickness=7)
                i+= 70
        
    
    # Display the frame.
    cv2.imshow('Face Filter', frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF    
    
    # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.                  
camera_video.release()
cv2.destroyAllWindows()

[-0.00357761  0.00026156 -0.0086434   0.00267525  0.00144864 -0.00239054
  0.00651442 -0.00480422]
0.4719406855922414
0.4762138002835191
0.4711765634383669
0.47345553766330056
0.4938271718791441
0.49605310873500397
0.4945912940330186
0.49890528072432877
[-0.00344853 -0.00825982  0.00072582  0.00300492  0.00236798  0.00717927
 -0.00180637 -0.00408547]
0.4000937083540389
0.4034904235612799
0.3993295862001644
0.4010374923399324
0.42198019464094155
0.42364619315975316
0.42274431679481606
0.426193128310416
[-5.39436648e-04 -1.55306964e-03 -2.27468300e-02  1.87462235e-05
 -2.32022802e-02 -2.21886472e-02 -9.94886771e-04 -2.37604630e-02]
0.3021803679047188
0.31667941084197837
0.2976174670430111
0.3039846678709637
0.5866079885287889
0.5966216770503475
0.5911708893904966
0.6168457338823095
[-0.00487349 -0.00126787 -0.01033536  0.00138705 -0.00046919 -0.00407481
  0.00499267 -0.00672974]
0.47273293607278705
0.4778539547962765
0.4727400279211187
0.47545384341717084
0.49539391104478187
0.4981030984

  2.76720094e-03  6.68524046e-03 -1.61348617e-03 -3.95750545e-03]
0.40591023186826236
0.4095750746237189
0.40606608949658146
0.40800351521138734
0.4326224795049113
0.4344856559427926
0.4324666218765922
0.43616864583855713
[-0.02344144 -0.02081932 -0.00044102 -0.0228102   0.00281233  0.00019022
 -0.02018808  0.0021811 ]
0.3054892703708282
0.31920647860273654
0.30632700512304323
0.31270661461508215
0.6031337169808472
0.6124780144728215
0.6022959822286321
0.6259754566725417
[ 0.00281847  0.00746365 -0.00219008 -0.00371397 -0.00407733 -0.00872251
  0.00093122  0.00245511]
0.47802298942900706
0.4823426771841018
0.47842486201160345
0.48056651159922303
0.5057658678360233
0.5078948779511866
0.5053639952534268
0.5097946011461696
[-0.00503316 -0.00905997 -0.00043586  0.00147909  0.00204959  0.0060764
 -0.00254771 -0.00446266]
0.40668539586327906
0.41052694527964684
0.407068131656228
0.4091436025099431
0.43362796588566443
0.43572625096131723
0.4332452300927155
0.43722418450812506
[-0.01275884 -0.

 -0.00150668 -0.00390334]
0.4038233953234649
0.4073982031906752
0.4042757940400053
0.406043327671044
0.43395713037196665
0.4356568535472362
0.43350473165542625
0.43714315207758686
[-0.01541125 -0.00212737 -0.01861089  0.0006287   0.01071295 -0.00257094
  0.01391258 -0.00532701]
0.4437127611285251
0.4496118031309866
0.4444667589894258
0.4470376128948143
0.4772724465142252
0.4797922132276681
0.4765184486533245
0.48260957065263166
[ 0.00101705  0.00214012 -0.0212946   0.00034076 -0.02084783 -0.02197089
  0.00146383 -0.02017154]
0.3020340302665254
0.31471446885230975
0.3041954908011074
0.3101396818474238
0.6010088541207211
0.6099449693686287
0.5988473935861391
0.6210669327499054
[-0.00813496 -0.00374291 -0.00331908 -0.00154582  0.00766212  0.00327006
  0.00284624  0.00107297]
0.4767328403844134
0.4807248093345097
0.4774379907203227
0.4794839108696426
0.5047783766124304
0.5068382393493889
0.5040732262765211
0.5081902193786956
[-0.0038311  -0.00806718  0.00050533  0.00269571  0.00279604  0.0

  0.00293711 -0.01849997]
0.3030115096772339
0.3135087496503581
0.3057995158011042
0.31091673977612766
0.6003094028203967
0.609322946848873
0.5975213966965264
0.6171576895379608
[-0.00717743 -0.00267762 -0.00248756 -0.00060394  0.00858575  0.00408593
  0.00389588  0.00201226]
0.4764161634580958
0.4799487078234369
0.4769889171503382
0.4787464138477698
0.5043279966526344
0.5060152241315025
0.5037552429603919
0.5073184225110295
[-0.00313012 -0.00725054  0.00105315  0.00337513  0.003438    0.00755841
 -0.00074528 -0.00306726]
0.4040736344148527
0.4072738670217391
0.4046872990851125
0.4062737825975754
0.43436972825903397
0.43591259916594505
0.4337560635887743
0.4370303538552331
[-0.01589424 -0.0032464  -0.01884879 -0.00018559  0.00950769 -0.00314015
  0.01246225 -0.00620095]
0.4441205582115777
0.450252222912678
0.4451160586766657
0.44834688283029234
0.4771418375222126
0.4804142581870301
0.47614633705712456
0.48252721491929196
[ 0.00237022  0.00348202 -0.01975028  0.0016921  -0.01931659 -0.0

  0.00266964 -0.01873107]
0.30433250954700897
0.3155959574385133
0.30727406472353425
0.3128026962102258
0.6001967738852855
0.6089094742136119
0.5972552187087602
0.6172055553154825
[ 0.00339361  0.0079747  -0.00127064 -0.00318973 -0.00327288 -0.00785397
  0.00139136  0.00331045]
0.4779288354094327
0.4818156844999733
0.4783839426599769
0.48032011112980055
0.5057250122876177
0.5076455902658614
0.5052699050370735
0.509251947580741
[-0.00362058 -0.00788315  0.00057411  0.00288854  0.00282066  0.00708323
 -0.00137402 -0.00368846]
0.4066966830130273
0.41008018075234176
0.4071517902635715
0.4088750879173091
0.4344928598912123
0.4362233744293841
0.434037752640668
0.437539437053253
[-0.01570997 -0.00240277 -0.01834901  0.00016318  0.01083134 -0.00247586
  0.01347038 -0.00504181]
0.4446748703324035
0.4505272182029242
0.445465890077397
0.44814349720535995
0.4774940002126333
0.4802239461734956
0.47670298046763976
0.48283490936748086
[ 0.00128881  0.00218631 -0.02085544  0.00061388 -0.02063288 -0.02

...

[-0.01766132 -0.02767927 -0.01730676 -0.01300322 -0.0226666  -0.01264866
 -0.02302116 -0.02732471]
0.44042815442014155
0.44642320877887015
0.4432728914592368
0.44735747450028845
0.4631691374142761
0.4717110231209182
0.46032440037518085
0.4708752634688887
[0.02626216 0.00965383 0.02532159 0.02335084 0.00580194 0.02241027
 0.00674251 0.00871326]
0.3910011076149532
0.38665397787949396
0.39319561904511247
0.38755322373580425
0.4089754003197988
0.40595563069016
0.4067808888896396
0.4051107813868037
[ 0.01453123  0.00262932 -0.01331313  0.02930313 -0.01044313  0.00145877
  0.01740122 -0.02521503]
0.4135771637525522
0.41547926270985885
0.41894152502627474
0.41746874698967446
0.444326390555118
0.44787652955499085
0.43896202928139544
0.4461143858967992
[ 0.02071889 -0.03119457  0.00361243  0.02153926 -0.04748066  0.0044328
 -0.0303742  -0.04830103]
0.30988059403234314
0.3091070445507121
0.32190976779766034
0.3153876981299833
0.5016620588397396
0.5339276015474855
0.4896328850744223
0.53140536363

 0.00811093 0.01053449]
0.3885762127545832
0.38336077747616454
0.3922600764702472
0.38593497693019935
0.40912385047850597
0.4057772836148945
0.405439986762842
0.403237190702692
[ 0.01283152  0.0009615  -0.01415214  0.02636435 -0.01248933 -0.00061931
  0.01449433 -0.02602216]
0.4148868829316523
0.41709529603164897
0.42262299673454673
0.4223895223361254
0.446232280346109
0.45112503560422323
0.4384961665432146
0.445940322262124
[ 0.02349987 -0.02817448  0.00759984  0.02437059 -0.04320378  0.00847057
 -0.02730375 -0.0440745 ]
0.3060702132425263
0.30248938175328904
0.3235071681633359
0.31561122704759387
0.49349401016658456
0.5233515826977579
0.47605705524577496
0.5120163510641986
[-0.01687646 -0.026746   -0.01660013 -0.01218504 -0.02177825 -0.01190871
 -0.02205458 -0.02646967]
0.44072290418449034
0.44613545899013707
0.4442610823177199
0.44792897981429447
0.46405757053716157
0.47223261536923
0.46051939240393197
0.4705203002948861
[0.02635457 0.00998103 0.0257839  0.02343161 0.0064874  0.0228

  0.01468094 -0.01982381]
0.5290771396211288
0.5385126772026414
0.526069097824319
0.5331884760358178
0.5536467421947848
0.5570922515572839
0.5566547839915946
0.5624785892893318
[-0.03362738 -0.0001267  -0.01358318 -0.03430477  0.01924012 -0.01426056
 -0.00080408  0.01991751]
0.4287281799811318
0.4547440644489315
0.4205700060170568
0.44080427581051806
0.6568433375153719
0.6468136718047852
0.665001511479447
0.6622205892750846
[0.00577798 0.02124862 0.00905078 0.00990916 0.02865261 0.01318196
 0.02537981 0.02452143]
0.5660068664931533
0.5636314272674006
0.5655937568658449
0.5619167586760314
0.5878417591334649
0.5794592399369229
0.5882548687607734
0.581200856092886
[-0.01605048 -0.00782629 -0.01544002 -0.02132672 -0.01249207 -0.02071626
 -0.01310254 -0.00721583]
0.507077605470392
0.516134049285921
0.5066644958430836
0.5144938286062437
0.5289124981107036
0.5327993270016335
0.5293256077380121
0.5344694773443158
[ 1.78629960e-05  1.62540561e-03 -2.50061518e-02  1.62972585e-02
 -7.11921369e-03

[0.01506214 0.03500772 0.01244609 0.01234519 0.02967472 0.00972914
 0.03229077 0.03239167]
0.5822092702766223
0.5775057827657172
0.5809988794402392
0.5749359283225444
0.5971122538440855
0.5855844889548697
0.5983226446804686
0.5881563851047679
[-0.03121324 -0.01741607 -0.03188185 -0.02670953 -0.01358096 -0.02737814
 -0.01291235 -0.01808467]
0.5287623823824157
0.5420571294069954
0.5273502597399687
0.539061564846574
0.5460417740484304
0.5521833823600376
0.5474538966908773
0.5551840112798077
[-0.00732121 -0.02950505  0.01583971  0.00631378  0.00729086  0.0294747
 -0.01587006 -0.00634413]
0.5629153313160786
0.5726070690318497
0.5603331641984614
0.5670261150446213
0.58353642334
0.5827446594225827
0.5861185904576173
0.5883326928366996
[-0.05113222  0.01391357 -0.03289267 -0.05180915  0.03147618 -0.03356961
  0.01323663  0.03215312]
0.4978759638792879
0.5399909585056951
0.4919450487810107
0.5268360124642059
0.6595241425810119
0.6410406560675974
0.665455057679289
0.654287098439795
[0.01413075 0

[0.00951406 0.01277538 0.00767903 0.00295044 0.00437674 0.00111542
 0.00621177 0.01094035]
0.5184370051528403
0.5177943896883619
0.5176738751980501
0.5177563694069613
0.5402868477564564
0.5409849890814067
0.5410499777112465
0.5409905267392859
[0.01012335 0.00388427 0.01134835 0.0165558  0.01154172 0.01778079
 0.01031672 0.00510927]
0.45015732799340796
0.44798558450482256
0.4493941980386178
0.44785027696644425
0.472007170597024
0.470916417823136
0.47277030055181424
0.47101969579303504
[ 2.44438053e-02  1.86383369e-02  5.27943155e-03  2.50341375e-02
  6.42953089e-05  5.86976372e-03  1.92286690e-02 -5.26036865e-04]
0.36500318512747887
0.35925004648430764
0.3608168722326297
0.3578353363698363
0.6259344815380822
0.6290355326928919
0.6301207944329313
0.628391269162702
[0.00904787 0.01185016 0.00733925 0.00233043 0.0034241  0.00062181
 0.00513272 0.01014153]
0.5149290729238637
0.5145293794627069
0.5145338726216012
0.5149028901764053
0.5371100034214253
0.5383433171941425
0.5375052037236878
0.5

In [11]:
right_eye = cv2.imread('data/right_eye_cupcake.png')

filter_img_height, filter_img_width, _  = right_eye.shape

pts1 = np.array([[0, 0], [0, filter_img_height], [filter_img_width, filter_img_height], [filter_img_width, 0]])
pts2 = np.array([[0, 0], [0, filter_img_height], [filter_img_width, filter_img_height], [filter_img_width, 0]])

h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)

print(h)

print(filter_img_height, filter_img_width)
print()

[[ 1.00000000e+00  0.00000000e+00 -2.46139224e-14]
 [ 0.00000000e+00  1.00000000e+00 -1.47683534e-13]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
463 396



In [12]:
x = np.array([[[1], [2]], [[3], [4]]])
print(x.shape)
print(x)


x = np.repeat(x, 3, axis=2)
print(x.shape)

print(x)


(2, 2, 1)
[[[1]
  [2]]

 [[3]
  [4]]]
(2, 2, 3)
[[[1 1 1]
  [2 2 2]]

 [[3 3 3]
  [4 4 4]]]


In [13]:
# TODO
# - Résoudre le problème du filtre de bouche
# - Gérer l'offset
# - Mettre en perspective